See more at: https://pandas-gbq.readthedocs.io/en/latest/intro.html#logging

In [1]:
# Required files
import os
import sys
import pandas as pd
import pandas_gbq
import re

# Connect colab with google bigquery stuff
from google.colab import auth
from google.cloud import bigquery
from google.colab import files
from google.colab import drive

**Authenticate:** The line of code below ensures you are an authenticated user accessing the MIMIC database. You will need to rerun this each time you open the notebook.

In [2]:
auth.authenticate_user()

**Query Function:** This is a method that executes a desired SQL query on the database. If you want to run a query, you can use the function name below, which we named run_query()

In [3]:
!jupyter-kernelspec list
# This line activate R magic
%load_ext rpy2.ipython

Available kernels:
  ir         /usr/local/share/jupyter/kernels/ir
  python3    /usr/local/share/jupyter/kernels/python3


In [4]:
# TODO:
# Specify your project_id on Google Cloud
project_id='test-project-mimic'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [5]:
# Read csv-sheet with ICD9_codes
%%R
#install.packages("readr")
library(tidyverse)
library(readr)
ICD9_Codes <- read_csv("/content/drive/MyDrive/ICD9_Codes_for_Cohort_Alcohol - Sheet1.csv", col_types = "ccc") #%>% select("ICD9_Code") %>% pull()
ICD9_Codes


✔ tibble  3.1.7      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 

✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



 




Error: '/content/ICD9_Codes_for_Cohort_Alcohol - Sheet1.csv' does not exist.


RInterpreterError: ignored

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import numpy as np
import rpy2.robjects as robjects

#pd.DataFrame(robjects.globalenv['ICD9_Codes'])

ICD9_codes_by_cohort =pd.read_csv("/content/drive/MyDrive/ICD9_Codes_for_Cohort_Alcohol - Sheet1.csv")
#ICD9_array = np.array(robjects.globalenv['ICD9_Codes']).tolist()


In [10]:
unique_cohorts = list(set(ICD9_codes_by_cohort["Cohort_Name"]))
output_dict = {}

ICD9_dict = {x : ICD9_codes_by_cohort.loc[ICD9_codes_by_cohort["Cohort_Name"] == x,"ICD9_Code"] for x in unique_cohorts}

In [11]:
ICD9_array = ICD9_codes_by_cohort["ICD9_Code"].values

In [14]:
cohort = unique_cohorts[2]
cohort

'Alcohol'

In [48]:
for cohort in unique_cohorts:
  current_cohort_ICD9_array = ICD9_codes_by_cohort.loc[ICD9_codes_by_cohort["Cohort_Name"]== cohort, "ICD9_Code"].values

  # First query: Patients chosen by ICD9 code
  query_1 = '''
    SELECT DISTINCT dx.SUBJECT_ID, dx.HADM_ID, ICUSTAY_ID
    FROM `physionet-data.mimiciii_clinical.d_icd_diagnoses` AS icd
    INNER JOIN `physionet-data.mimiciii_clinical.diagnoses_icd` AS dx
      ON dx.ICD9_CODE = icd.ICD9_CODE
    INNER JOIN `physionet-data.mimiciii_clinical.icustays` AS icu
    ON (dx.SUBJECT_ID = icu.SUBJECT_ID AND dx.HADM_ID = icu.HADM_ID)
    WHERE dx.ICD9_CODE IN ({})
    '''
  placeholders = ', '.join(['"{}"'] * len(current_cohort_ICD9_array))
  query_1 = query_1.format(placeholders)
  query_1 = query_1.format(*current_cohort_ICD9_array)

  Cohort_hadm_ids_1 = (pandas_gbq.read_gbq(query_1, project_id=project_id, dialect = 'standard'))

  # Second query: Patients chosen by clinical note text pattern
  query_2 = '''
  SELECT DISTINCT ne.SUBJECT_ID, ne.HADM_ID, icu.ICUSTAY_ID
  FROM `physionet-data.mimiciii_notes.noteevents` AS ne
    INNER JOIN `physionet-data.mimiciii_clinical.icustays` AS icu
    ON (ne.SUBJECT_ID = icu.SUBJECT_ID AND ne.HADM_ID = icu.HADM_ID)
  WHERE
  FALSE
  AND REGEXP_CONTAINS(LOWER(TEXT), "")
    '''
  #placeholders = ', '.join(['"{}"'] * len(current_cohort_ICD9_array))
  #query_2 = query_2.format(placeholders)
  #query_2 = query_2.format(*current_cohort_ICD9_array)

  #Cohort_hadm_ids_2 = (pandas_gbq.read_gbq(query_2, project_id=project_id, dialect = 'standard'))

  Cohort_hadm_ids
  Cohort_hadm_ids.to_csv(cohort + ".csv", index = True)



Downloading: 100%|██████████| 824/824 [00:00<00:00, 5288.50rows/s]


In [49]:
Cohort_hadm_ids


,SUBJECT_ID,HADM_ID,ICUSTAY_ID
0,41984,176631,268581
1,25345,160873,214940
2,25345,180713,226343
3,56321,174240,297195
4,76803,189589,253435
...,...,...,...
3619,14844,140864,250220
3620,98813,170407,283274
3621,18430,169641,266427
3622,89854,110086,245825
